In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

O conjunto de dados CIFAR10 contém 60.000 imagens coloridas em 10 classes, com 6.000 imagens em cada classe. O conjunto de dados é dividido em 50.000 imagens de treinamento e 10.000 imagens de teste. As classes são mutuamente exclusivas e não há sobreposição entre elas.

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)

In [ ]:
train_images

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
train_images

## Verifique os dados

Para verificar se o conjunto de dados está correto, plote as 25 primeiras imagens do conjunto de treinamento e exiba o nome da classe abaixo de cada imagem.

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## Crie a base convolucional

Como entrada, uma CNN assume tensores de forma (altura da imagem, largura da imagem, canais de cores), ignorando o tamanho do lote. Se você é novo nessas dimensões, color_channels se refere a (R, G, B). Neste exemplo, você configurará nossa CNN para processar entradas de forma (32, 32, 3), que é o formato das imagens CIFAR. Você pode fazer isso passando o argumento input_shape para nossa primeira camada.

In [ ]:
# modela a etapa de convolução com o operador de convolução Conv2D, faz o maxpolling 
model = models.Sequential()
# aplica o operador de convolução para identificar os melhores feature detector
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
# o primeiro paremtro da função de convolução é a quantidade de filtros (kernels) o segundo é o tamanho da matriz de cada kernel (feture detector)
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Você pode ver que a saída de todas as camadas Conv2D e MaxPooling2D é um tensor de forma 3D (altura, largura, canais). As dimensões de largura e altura tendem a encolher à medida que você avança na rede. O número de canais de saída para cada camada do Conv2D é controlado pelo primeiro argumento (por exemplo, 32 ou 64).

In [ ]:
model.summary()

##Adiciona camadas densas na parte superior

Para concluir nosso modelo, você alimentará o último tensor de saída da base convolucional (de forma (4, 4, 64)) em uma ou mais camadas densas para realizar a classificação. 

Camadas densas recebem vetores como entrada (que são 1D), enquanto a saída atual é um tensor 3D. Primeiro, você achatará (ou desenrolará) a saída 3D para 1D e depois adicionará uma ou mais camadas densas na parte superior. 

O CIFAR possui 10 classes de saída, portanto, você usa uma camada Densa final com 10 saídas e uma ativação softmax.

In [ ]:
#a primeira linha realiza o flattening do resultado da camada de convolução, ou seja, transforma a matriz da ultima camada da convolução em um vetor
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

## Compilando e Treinando o modelo

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

## Avaliando o Modelo

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

Nossa CNN simples alcançou uma precisão de teste superior a 70%.